Se importan las librerias pertinentes y se monta el drive para poder traer los archivos a usar desde el drive

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import json
import ast
import nltk
from pandas import json_normalize
import re
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


se guardan las rutas en variables para poder comenzar con su extraccion



In [2]:
ruta_output_steam = '/content/drive/MyDrive/TP N1 Henry/conversion datos/output_steam_games.json'
ruta_aust_items = '/content/drive/MyDrive/TP N1 Henry/conversion datos/australian_users_items.json'
ruta_aust_reviews = '/content/drive/MyDrive/TP N1 Henry/conversion datos/australian_user_reviews.json'

# se definen 3 diccionaros vacíos donde se guardaran las lineas de cada Json
lista_json_steam = []
lista_json_items = []
lista_json_reviews = []

Se comienza con el proceso de extraccion iterando sobre cada fila de cada JSON y guardando cada esta dentro de de sus respectivos diccionarios para luego ser convertidos a DataFrames

In [3]:
with open(ruta_output_steam, 'r') as file:
    for line in file:
        try:
            item = json.loads(line)
            lista_json_steam.append(item)
        except json.JSONDecodeError as e:
            print(f"Error de decodificación en la línea: {line}")
            print(e)


## se convierte en un Dataframe
pd_output_steam = pd.DataFrame(lista_json_steam)
pd_output_steam.head()
# observamos que este DataFrame cuenta con muchos valores nulos
# los trataremos en breve

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
with open(ruta_aust_items, 'r', encoding='utf-8') as f_items:
    for line in f_items:
        try:
            item = ast.literal_eval(line)
            lista_json_items.append(item)
        except Exception as e:
            print(f"Error en la línea: {line}")
            print(f"Mensaje de error: {e}")



# Se convierte en un DataFrame
pd_aust_items = pd.DataFrame(lista_json_items)
pd_aust_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [5]:
with open(ruta_aust_reviews, 'r', encoding='utf-8') as f_reviews:
    for line in f_reviews:
        try:
            item = ast.literal_eval(line)
            lista_json_reviews.append(item)
        except Exception as e:
            print(f"Error en la línea: {line}")
            print(f"Mensaje de error: {e}")



# Se convierte en un Dataframe
pd_aust_reviews = pd.DataFrame(lista_json_reviews)
pd_aust_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


quitaremos toda fila que contenga todas sus columnas con valores nulos

In [6]:
# pero primero veremos cuanto porcentaje de nulos tiene cada Dataframe por columna
print(pd_output_steam.isnull().sum() /len(pd_output_steam) *100)

publisher       80.004982
genres          76.045498
app_name        73.321433
title           75.021794
url             73.319773
release_date    75.035909
tags            73.455104
reviews_url     73.321433
specs           73.876043
price           74.463033
early_access    73.319773
id              73.321433
developer       76.058782
dtype: float64


In [7]:
print(pd_aust_items.isnull().sum() /len(pd_aust_items) *100)

user_id        0.0
items_count    0.0
steam_id       0.0
user_url       0.0
items          0.0
dtype: float64


In [8]:
print(pd_aust_reviews.isnull().sum() /len(pd_aust_reviews) *100)

user_id     0.0
user_url    0.0
reviews     0.0
dtype: float64


con el proceso anterior nos dimos cuenta que solo el Dataframe de steam contiene filas con la condicion dicha


In [9]:
registros_eliminados_output_steam = len(pd_output_steam) - len(pd_output_steam.dropna(how='all'))
print(f"Registros a eliminar en pd_output_steam: {registros_eliminados_output_steam}")

Registros a eliminar en pd_output_steam: 88310


In [10]:
pd_output_steam.dropna(how='all', inplace = True)
pd_output_steam.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN


A continuacion realizaremos cambios en varias columnas del dataframe



Comenzaremos con completar y dar formato a la tabla de de juegos de steam

Valores nulos de:
- publisher
- genres
- url
- specs
se cambiaron por Desconocido

valores nulos de title y app_name:
- en casos donde uno solo sea nulo se pondra el valor del otro
- en casos donde ambos sean nulos se pondra Desconocido

en el caso de price, se toman todos los valores distintos son tratados individualmente segun el texto que tienen

se generan ids para quienes no tenian id

se cambiaron valores errones en release_date por 03-02-2000

In [11]:
pd_output_steam['publisher'] = pd_output_steam['publisher'].fillna('Desconocido')
pd_output_steam['publisher'] = pd_output_steam['publisher'].replace('-','Desconocido')
pd_output_steam['publisher'] = pd_output_steam['publisher'].replace('---','Desconocido')
pd_output_steam['genres'] = pd_output_steam['genres'].fillna('Desconocido')
pd_output_steam['app_name'] = pd_output_steam['app_name'].fillna(pd_output_steam['title'])
pd_output_steam['title'] = pd_output_steam['title'].fillna(pd_output_steam['app_name'])
pd_output_steam['app_name'] = pd_output_steam['app_name'].fillna('Desconocido')
pd_output_steam['title'] = pd_output_steam['title'].fillna('Desconocido')
pd_output_steam['url'] = pd_output_steam['url'].fillna('Desconocido')
pd_output_steam['specs'] = pd_output_steam['specs'].fillna('desconocido')
pd_output_steam['tags'] = pd_output_steam['tags'].fillna('etiquetas desconocidas')
pd_output_steam['reviews_url'] = pd_output_steam['reviews_url'].fillna('https://store.steampowered.com')
pd_output_steam['release_date'] = pd_output_steam['release_date'].fillna('2000-02-03')

pd_output_steam['id'] = pd.to_numeric(pd_output_steam['id'], errors='coerce')
max_id = pd_output_steam['id'].max()
no_id = pd_output_steam[pd_output_steam['id'].isna()]
nuevas_id = range(int(max_id) + 1, int(max_id) + 1 + len(no_id))
pd_output_steam.loc[pd_output_steam['id'].isna(), 'id'] = nuevas_id

Este codigo se usa para obtener los valores no numericos dentro de price asi luego podemos cambiarlos

In [12]:

non_numeric_prices = pd_output_steam.loc[pd_output_steam['price'].apply(lambda x: isinstance(x, str) and not x.replace('.', '', 1).isdigit())]
valores_distintos = non_numeric_prices['price'].unique()
valores_distintos

array(['Free To Play', 'Free to Play', 'Free', 'Free Demo',
       'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo',
       'Free Mod', 'Install Theme', 'Third-party', 'Play Now',
       'Free HITMAN™ Holiday Pack', 'Play the Demo',
       'Starting at $499.00', 'Starting at $449.00', 'Free to Try',
       'Free Movie', 'Free to Use'], dtype=object)

In [13]:
pd_output_steam['price'] = pd_output_steam['price'].replace({
    'Free To Play': 0,
    'Free to Play':0,
    'Free To Play':0,
    'Free to Try':0,
    'Free': 0,
    'Free Demo': 0,
    'Play for Free!': 0,
    'Install Now': 0,
    'Play WARMACHINE: Tactics Demo': 0,
    'Free Mod': 0,
    'Install Theme': 0,
    'Third-party': 0,
    'Play Now': 0,
    'Free HITMAN™ Holiday Pack': 0,
    'Play the Demo': 0,
    'Free Movie': 0,
    'Free to Use': 0,
    'Starting at $499.00': 500,
    'Starting at $449.00': 450
})

Cambiamos los formatos de la tabla

In [14]:
pd_output_steam.dtypes

publisher        object
genres           object
app_name         object
title            object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price           float64
early_access     object
id              float64
developer        object
dtype: object

In [15]:
pd_output_steam['release_date'] = pd.to_datetime(pd_output_steam['release_date'], errors='coerce')
pd_output_steam['early_access'] = pd_output_steam['early_access'].astype(bool)
pd_output_steam['price'] = pd_output_steam['price'].astype(float)

<ipython-input-15-7d3ac6e7bfe4>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  pd_output_steam['release_date'] = pd.to_datetime(pd_output_steam['release_date'], errors='coerce')


Convertimmos el Dataframe en limpio en otro llamado Steam_Games, y lo tenemos listo para convertirlo en csv

In [16]:
steam_games = pd_output_steam
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400.0,彼岸领域
88314,Desconocido,Desconocido,Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,2000-02-03,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870.0,"xropi,stev3ns"


trabajaremmos sobre el DataFrame de las reviews, desglosaremos la columna reviews, quitaremos lo que no nos sirva y aplicaremos un lenguaje de procesamiento natural para obtener informacion necesaria

In [17]:

expanded_df = pd_aust_reviews.explode('reviews')
expanded_df

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [18]:
normalized_df = json_normalize(expanded_df['reviews'])
normalized_df

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
59328,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,,Posted July 20.,,730,No ratings yet,True,:D


In [19]:
result_df = pd.concat([expanded_df.reset_index(drop=True), normalized_df], axis=1)
result_df = result_df.drop_duplicates(subset=['user_id', 'user_url', 'item_id', 'recommend', 'review'])
columnas = ['reviews','funny','last_edited']
result_df = result_df.drop(columns=columnas)
result_df


,user_id,user_url,posted,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,No ratings yet,True,:D


Extraemos las interacciones con la review de otros usuarios


In [20]:
def extractor_porcentaje(row):
    helpful_text = str(row['helpful'])  # Convertir a cadena para manejar el caso cuando no sea una cadena
    match = re.search(r'\((\d+)%\)', helpful_text)
    return int(match.group(1)) if match else 0

result_df['rev_utility'] = result_df.apply(extractor_porcentaje, axis=1)

result_df.drop(columns='helpful')

,user_id,user_url,posted,item_id,recommend,review,rev_utility
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,0
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,75
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,0
...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,True,a must have classic from steam definitely wort...,0
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,True,this game is a perfect remake of the original ...,0
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,50
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,True,:D,0


In [21]:
result_df['review'].dropna(inplace=True)

aplicamos la funcion SentimentIntensityAnalizer de la libreria **nltk**

In [22]:
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer


sia = SentimentIntensityAnalyzer()
umbral_negativo = -0.1
umbral_positivo = 0.1


for index, row in result_df.iterrows():
    if pd.notnull(row['review']):
        sentiment_score = sia.polarity_scores(row['review'])['compound']

        if sentiment_score < umbral_negativo:
            result_df.at[index, 'sentimiento'] = 0
        elif sentiment_score > umbral_positivo:
            result_df.at[index, 'sentimiento'] = 2
        else:
            result_df.at[index, 'sentimiento'] = 1
    else:
        result_df.at[index, 'sentimiento'] = 1


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


al igual que con steam_games, aqui convertimos pd_aust_reviews en un Dataframe pasado en limpio

In [23]:
steam_reviews=result_df

Ahora trabajaremos en el DataFrame de items


In [24]:
pd_aust_items.isnull().any() # Esta comppleto

user_id        False
items_count    False
steam_id       False
user_url       False
items          False
dtype: bool

In [25]:
pd_aust_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Desglosaremos el diccionario presente en la columna items y crearemos nuevas filas y columnas para sus datos


In [26]:
items_data = []
for index, row in pd_aust_items.iterrows():
  user_id = row['user_id']
  user_url = row['user_url']
  for item_value in row['items']:
    item_data = {'user_id': user_id, 'user_url': user_url, **item_value}
    items_data.append(item_data)
pd_aust_items = pd.DataFrame(items_data)
pd_aust_items

,user_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...,...
5153204,76561198329548331,http://steamcommunity.com/profiles/76561198329...,346330,BrainBread 2,0,0
5153205,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0,0
5153206,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3,3
5153207,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4,4


In [27]:
steam_items = pd_aust_items

convertimos los 3 DataFrames en CSV para su posterior tratado

In [28]:
steam_games.to_csv('steam_games.csv', index=False)

steam_items.to_csv('steam_items.csv', index=False)

steam_reviews.to_csv('steam_reviews.csv', index=False)